In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI RAG Engine with Pinecone

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/rag-engine/rag_engine_pinecone.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Frag-engine%2Frag_engine_pinecone.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/rag-engine/rag_engine_pinecone.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/rag-engine/rag_engine_pinecone.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | |
|-|-|
| Author(s) | [Darshan Mehta](https://github.com/darshanmehta17) |

## Overview

This notebook illustrates how to use [Vertex AI RAG Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/rag-overview) with [Pinecone](https://www.pinecone.io/) as a vector database.

For more information, refer to the [official documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/use-pinecone).

For more details on RAG corpus/file management and detailed support please visit [Vertex AI RAG Engine API](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/rag-api)

## Get started

### Install Vertex AI SDK and other required packages


In [1]:
%pip install --upgrade --user --quiet google-cloud-aiplatform google-cloud-secret-manager "pinecone-client[grpc]"

DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/fsspec-2024.3.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google_cloud_documentai_toolbox-0.12.2a0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google_cloud_documentai_toolbox-0.11.1a0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://gi

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
import os

import vertexai

PROJECT_ID = "[your-project-id]"  # @param {type:"string", isTemplate: true}
if PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

vertexai.init(project=PROJECT_ID, location=LOCATION)

## (Optional) Setup Pinecone Instance

In this section, we have some helper methods to help you setup your Pinecone instance.

This section is not required if you already have a Pinecone instance ready to use.

### Initialize the Pinecone Python client

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec, PodSpec

# Set API Key
# Copy this value from your pinecone.io console
PINECONE_API_KEY = ""  # @param {type:"string"}

pc = Pinecone(api_key=PINECONE_API_KEY)

### Create a Pinecone instance

Use the below section to create Pinecone indexes with a spec of your choice (serverless or pod). Read more about Pinecone indexes [here](https://docs.pinecone.io/guides/indexes/create-an-index).

#### Serverless Index

In [ ]:
# Index Configs
INDEX_NAME = ""  # @param {type:"string"}

# Choose a distance metric
DISTANCE_METRIC = (
    "cosine"  # @param ["cosine", "euclidean", "dotproduct"] {allow-input: true}
)

# This number should match the dimension size of the embedding model you choose
# for your RAG Corpus.
EMBEDDING_DIMENSION_SIZE = 768  # @param {"type":"number","placeholder":"768"}

CLOUD_PROVIDER = "gcp"  # @param ["gcp", "aws", "azure"] {allow-input: true}

# Choose the right region for your cloud provider of choice
# Refer https://docs.pinecone.io/guides/indexes/understanding-indexes#cloud-regions
CLOUD_REGION = "us-central1"  # @param {type:"string"}


# Create the index
pc.create_index(
    name=INDEX_NAME,
    dimension=EMBEDDING_DIMENSION_SIZE,
    metric=DISTANCE_METRIC,
    spec=ServerlessSpec(cloud=CLOUD_PROVIDER, region=CLOUD_REGION),
    deletion_protection="disabled",
)

#### Pod-based Index


In [ ]:
# Index Configs
INDEX_NAME = ""  # @param {type:"string"}

# Choose a distance metric
DISTANCE_METRIC = (
    "cosine"  # @param ["cosine", "euclidean", "dotproduct"] {allow-input: true}
)

# This number should match the dimension size of the embedding model you choose
# for your RAG Corpus.
EMBEDDING_DIMENSION_SIZE = 768  # @param {"type":"number","placeholder":"768"}

# Choose the right environment for your cloud provider of choice
# Refer https://docs.pinecone.io/guides/indexes/understanding-indexes#pod-environments
ENVIRONMENT = "us-central1-gcp"  # @param {type:"string"}

# Choose the pod type
# Refer to https://docs.pinecone.io/guides/indexes/understanding-indexes#pod-based-indexes
POD_TYPE = "p1.x1"  # @param {type:"string"}

# Explore all the parameters you can play with for creating a pod index by
# following this page:
# https://docs.pinecone.io/reference/api/2024-07/control-plane/create_index
pc.create_index(
    name=INDEX_NAME,
    dimension=EMBEDDING_DIMENSION_SIZE,
    metric=DISTANCE_METRIC,
    spec=PodSpec(
        environment=ENVIRONMENT,
        pod_type=POD_TYPE,
        pods=1,
        metadata_config={
            "indexed": ["file_id"]  # This field is required for pod-based indexes.
        },
    ),
    deletion_protection="disabled",
)

## (Optional) Setup Secret Manager

This section helps you add your Pinecone API key to your Google Cloud Secret Manager. This section is not required is you already have a secret with the API key ready to use.

In [ ]:
# Google Cloud project ID and the Pinecone API key will be used from the above sections.
# Choose your secret ID
SECRET_ID = ""  # @param {type:"string"}

In [ ]:
from google.cloud import secretmanager

client = secretmanager.SecretManagerServiceClient()

# Create the secret.
secret = client.create_secret(
    parent=client.common_project_path(PROJECT_ID),
    secret_id=SECRET_ID,
    secret=secretmanager.Secret(
        replication=secretmanager.Replication(automatic=secretmanager.Replication.Automatic())
    )
)

# Add API key to the secret payload.
secret_version = client.add_secret_version(
    parent=secret.name,
    payload=secretmanager.SecretPayload(data=PINECONE_API_KEY.encode("UTF-8"))
)

print(f"Created secret and added first version: {secret_version.name}")

## Get Service Account Information

In [ ]:
project_numbers = !gcloud projects list --filter="PROJECT_ID={PROJECT_ID}" --format="value(PROJECT_NUMBER)"
PROJECT_NUMBER = project_numbers[0]

# Construct your Knowledge Engine service account name
# Do not update this string since this is the name assigned to your service account.
SERVICE_ACCOUNT = (
    f"service-{PROJECT_NUMBER}@gcp-sa-vertex-rag.iam.gserviceaccount.com"
)

## Create a RAG corpus

In [ ]:
from vertexai.preview import rag
from vertexai.preview.generative_models import GenerativeModel, Tool

### First RAG Corpus Only

If this is the first RAG Corpus in your project, you might not be able to provide the Knowledge Engine service account access to your secret resource. So this section first creates a RAG Corpus with an empty Pinecone config. With this call, the service account for your project is provisioned.

Next, it assigns the service account permissions to read your secret. Finally, it updates your RAG Corpus with the Pinecone index name and the secret resource name.

#### Create a RAG Corpus without Pinecone information

In [ ]:
# Start with empty Pinecone config.
vector_db = rag.Pinecone()

# Name your corpus
DISPLAY_NAME = ""  # @param  {type:"string"}

# Create RAG Corpus
rag_corpus = rag.create_corpus(display_name=DISPLAY_NAME, vector_db=vector_db)
print(f"Created RAG Corpus resource: {rag_corpus.name}")

#### Grant your Knowledge Engine service account access to your API key secret

In [ ]:
!gcloud secrets add-iam-policy-binding {secret.name} \
  --member="serviceAccount:{SERVICE_ACCOUNT}" \
  --role="roles/secretmanager.secretAccessor"

#### Call the `UpdateRagCorpus` API to add the Pinecone index name and API key secret to your RAG Corpus

In [ ]:
# Name of your created Pinecone Index
PINECONE_INDEX_NAME = ""  # @param  {type:"string"}

# Construct your updated Pinecone config.
vector_db = rag.Pinecone(index_name=PINECONE_INDEX_NAME, api_key=secret_version.name)

updated_rag_corpora = rag.update_corpus(
    corpus_name=rag_corpus.name, vector_db=vector_db
)
print(f"Updated RAG Corpus: {rag_corpus.name}")

### Second RAG Corpus Onwards

In this case, since your service account is already generated, you can directly grant it permissions to access your secret resource containing the Pinecone API key as covered by the steps in the Setup Secret Manager section.


#### Grant your Knowledge Engine service account access to your API key secret

In [ ]:
!gcloud secrets add-iam-policy-binding {secret.name} \
  --member="serviceAccount:{SERVICE_ACCOUNT}" \
  --role="roles/secretmanager.secretAccessor"

#### Create a RAG Corpus with Pinecone information

In [ ]:
# Name of your created Pinecone Index
PINECONE_INDEX_NAME = ""  # @param  {type:"string"}
# Construct your Pinecone config.
vector_db = rag.Pinecone(index_name=PINECONE_INDEX_NAME, api_key=secret_version.name)

# Name your corpus
DISPLAY_NAME = ""  # @param  {type:"string"}

# Create RAG Corpus
rag_corpus = rag.create_corpus(display_name=DISPLAY_NAME, vector_db=vector_db)
print(f"Created RAG Corpus resource: {rag_corpus.name}")

## Upload a file to the corpus

In [ ]:
%%writefile test.txt

Here's a demo for Pinecone RAG.

In [ ]:
rag_file = rag.upload_file(
    corpus_name=rag_corpus.name,
    path="test.txt",
    display_name="test.txt",
    description="my test",
)
print(f"Uploaded file to resource: {rag_file.name}")

## Import files from Google Cloud Storage

Remember to grant "Viewer" access to the "Vertex RAG Data Service Agent" (with the format of `service-{project_number}@gcp-sa-vertex-rag.iam.gserviceaccount.com`) for your Google Cloud Storage bucket

In [ ]:
GCS_BUCKET = ""  # @param {type:"string", "placeholder": "your-gs-bucket"}

response = rag.import_files(  # noqa: F704
    corpus_name=rag_corpus.name,
    paths=[GCS_BUCKET],
    chunk_size=512,
    chunk_overlap=50,
)

In [ ]:
# Check the files just imported. It may take a few seconds to process the imported files.
rag.list_files(corpus_name=rag_corpus.name)

## Import files from Google Drive

Eligible paths can be:

- `https://drive.google.com/drive/folders/{folder_id}`
- `https://drive.google.com/file/d/{file_id}`

Remember to grant "Viewer" access to the "Vertex RAG Data Service Agent" (with the format of `service-{project_number}@gcp-sa-vertex-rag.iam.gserviceaccount.com`) for your Drive folder/files.


In [ ]:
FILE_ID = ""  # @param {type:"string", "placeholder": "your-file-id"}
FILE_PATH = f"https://drive.google.com/file/d/{FILE_ID}"

rag.import_files(
    corpus_name=rag_corpus.name,
    paths=[FILE_PATH],
    chunk_size=1024,
    chunk_overlap=100,
)

In [ ]:
# Check the files just imported. It may take a few seconds to process the imported files.
rag.list_files(corpus_name=rag_corpus.name)

## Use your RAG Corpus to add context to your Gemini queries

When retrieved contexts similarity distance < `vector_distance_threshold`, the contexts (from `RagStore`) will be used for content generation.

In [ ]:
rag_resource = rag.RagResource(
    rag_corpus=rag_corpus.name,
)

rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[rag_resource],  # Currently only 1 corpus is allowed.
            similarity_top_k=10,
            vector_distance_threshold=0.4,
        ),
    )
)

rag_model = GenerativeModel("gemini-1.5-flash", tools=[rag_retrieval_tool])

In [ ]:
GENERATE_CONTENT_PROMPT = "What is RAG and why it is helpful?"  # @param {type:"string"}

response = rag_model.generate_content(GENERATE_CONTENT_PROMPT)

response

## Using other generation API with Rag Retrieval Tool

The retrieved contexts can be passed to any SDK or model generation API to generate final results.

In [ ]:
RETRIEVAL_QUERY = "What is RAG and why it is helpful?"  # @param {type:"string"}

rag_resource = rag.RagResource(
    rag_corpus=rag_corpus.name,
    # Need to manually get the ids from rag.list_files.
    # rag_file_ids=[],
)

response = rag.retrieval_query(
    rag_resources=[rag_resource],  # Currently only 1 corpus is allowed.
    text=RETRIEVAL_QUERY,
    similarity_top_k=10,
)

# The retrieved context can be passed to any SDK or model generation API to generate final results.
retrieved_context = " ".join(
    [context.text for context in response.contexts.contexts]
).replace("\n", "")

retrieved_context

## Cleaning up

Clean up resources created in this notebook.

In [ ]:
delete_rag_corpus = False  # @param {type:"boolean"}

if delete_rag_corpus:
    rag_corpus_list = rag.list_corpora()
    for rag_corpus in rag_corpus_list:
        rag.delete_corpus(name=rag_corpus.name)